In [12]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import numpy as np


customers = pd.read_csv('Customers.csv')
products = pd.read_csv('Products.csv')
transactions = pd.read_csv('Transactions.csv')


transactions_merged = transactions.merge(products, on='ProductID', how='left')
transactions_merged = transactions_merged.merge(customers, on='CustomerID', how='left')


customer_profiles = transactions_merged.groupby('CustomerID').agg(
    total_spent=('TotalValue', 'sum'),
    total_transactions=('TransactionID', 'count'),
    unique_products=('ProductID', pd.Series.nunique)
).reset_index()


scaler = MinMaxScaler()
customer_profiles_scaled = scaler.fit_transform(customer_profiles[['total_spent', 'total_transactions', 'unique_products']])


similarity_matrix = cosine_similarity(customer_profiles_scaled)


customer_ids = customer_profiles['CustomerID'].tolist()
lookalike_data = {}

for idx, customer_id in enumerate(customer_ids[:20]):
    similarities = similarity_matrix[idx]
    top_indices = np.argsort(similarities)[::-1][1:4]
    lookalike_data[customer_id] = [(customer_ids[i], round(similarities[i], 2)) for i in top_indices]


lookalike_df = pd.DataFrame({
    "CustomerID": lookalike_data.keys(),
    "Recommendations": [str(v) for v in lookalike_data.values()]
})

lookalike_df.to_csv('Lookalike.csv', index=False)
print("Lookalike recommendations saved to Lookalike.csv")


Lookalike recommendations saved to Lookalike.csv
